<a href="https://colab.research.google.com/github/annamucha3/BSUclasses_AM/blob/main/Healthcare_modeling_20240506.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Updated on 5/5 adding in XGBoost & comparing model metrics with the others code that includes APR DRG Description, CCSR Procedure Description, and CCSR Diagnosis Description. This modeling does not include those three columns.

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Code using: health_data_clean_updated_20240418

In [ ]:
file_path = "/content/health_data_clean_updated_20240418.xlsx"
health_data = pd.read_excel(file_path)

# These 3 columns need to be fixed (APR DRG Description, CCSR Procedure Description, and CCSR Diagnosis Description).

In [ ]:
unique_CCSR_diagnosis = health_data['CCSR Diagnosis Description'].unique().tolist()
unique_CCSR_diagnosis

['COMPLICATION OF OTHER SURGICAL OR MEDICAL CARE, INJURY, INITIAL ENCOUNTER',
 'FRACTURE OF THE NECK OF THE FEMUR (HIP), INITIAL ENCOUNTER',
 'OCCLUSION OR STENOSIS OF PRECEREBRAL OR CEREBRAL ARTERIES WITHOUT INFARCTION',
 'DIVERTICULOSIS AND DIVERTICULITIS',
 'CYSTIC FIBROSIS',
 'ACUTE HEMORRHAGIC CEREBROVASCULAR DISEASE',
 'CONDUCTION DISORDERS',
 'PERITONITIS AND INTRA-ABDOMINAL ABSCESS',
 'ASTHMA',
 'COMPLICATIONS SPECIFIED DURING CHILDBIRTH',
 'ACUTE BRONCHITIS',
 'VIRAL INFECTION',
 'RESPIRATORY FAILURE; INSUFFICIENCY; ARREST',
 'SEQUELA OF HEMORRHAGIC CEREBROVASCULAR DISEASE',
 'LIVEBORN',
 'DIGESTIVE CONGENITAL ANOMALIES',
 'SEQUELA OF CEREBRAL INFARCTION AND OTHER CEREBROVASCULAR DISEASE',
 'ACUTE AND UNSPECIFIED RENAL FAILURE',
 'PARALYSIS (OTHER THAN CEREBRAL PALSY)',
 'ACUTE MYOCARDIAL INFARCTION',
 'CORONARY ATHEROSCLEROSIS AND OTHER HEART DISEASE',
 'FRACTURE OF THE SPINE AND BACK, INITIAL ENCOUNTER',
 'OBESITY',
 'NONRHEUMATIC AND UNSPECIFIED VALVE DISORDERS',
 'HYPERTEN

In [ ]:
unique_CCSR_procedure = health_data['CCSR Procedure Description'].unique().tolist()
unique_CCSR_procedure

['SUBCUTANEOUS TISSUE AND FASCIA EXCISION',
 'HIP ARTHROPLASTY',
 'CAROTID ENDARTERECTOMY AND STENTING',
 'COLONOSCOPY AND PROCTOSCOPY WITH BIOPSY',
 'VENOUS AND ARTERIAL CATHETER PLACEMENT',
 'FLUOROSCOPIC ANGIOGRAPHY (EXCLUDING CORONARY)',
 'PACEMAKER AND DEFIBRILLATOR PROCEDURES',
 'PARACENTESIS',
 'ISOLATION PROCEDURES',
 'SPONTANEOUS VAGINAL DELIVERY',
 'AIRWAY INTUBATION',
 'CEREBROSPINAL FLUID SHUNT PROCEDURES',
 'ASSISTED VAGINAL DELIVERY',
 'CIRCUMCISION',
 'UPPER GI THERAPEUTIC PROCEDURES, NEC (OPEN AND LAPAROSCOPIC)',
 'PHYSICAL, OCCUPATIONAL, AND RESPIRATORY THERAPY TREATMENT',
 'LUMBOSACRAL NERVE DECOMPRESSION',
 'MEASUREMENT DURING CARDIAC CATHETERIZATION',
 'CARDIAC AND CORONARY FLUOROSCOPY',
 'SPINE FUSION',
 'GASTRO-JEJUNAL BYPASS (INCLUDING BARIATRIC)',
 'ADMINISTRATION OF NUTRITIONAL AND ELECTROLYTIC SUBSTANCES',
 'SKIN EXCISION AND DEBRIDEMENT',
 'PERCUTANEOUS CORONARY INTERVENTIONS (PCI)',
 'ULTRASONOGRAPHY',
 'PHOTOTHERAPY',
 'CESAREAN SECTION',
 'NEPHRECTOMY AND 

# ERROR HERE. THIS NEEDS TO BE FIXED.

In [ ]:
unique_APR = health_data['APR DRG Description'].unique().tolist()
unique_APR

['Infections',
 'Musculoskeletal Disorders',
 'Other Diseases and Conditions Diseases and Conditions',
 'Digestive System Disorders',
 'Cardiovascular Diseases',
 nan,
 'Respiratory Diseases',
 'Maternal and Obstetric Conditions',
 'Nervous System Disorders',
 'Endocrine Disorders',
 'Injury and Trauma',
 'Genitourinary Disorders',
 'Cancer and Malignant Neoplasms',
 'Mental Health Disorders',
 'Hematological Disorders',
 'Mental health Disorders',
 'Other Diseases and Conditions']

In [ ]:
health_data.columns

Index(['Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition', 'CCSR Diagnosis Description',
       'CCSR Procedure Description', 'APR DRG Description',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs',
       'General_MDC'],
      dtype='object')

## I am making a subset of the data not including the CCSR columns since those have so many unique values. I took out medical description, since this didnt seem very useful for us as it didnt tell us much. Medical vs surgical.. not important? what does that mean to us?

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
health_data_copy = ['Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition',  'APR Severity of Illness Description', 'APR Risk of Mortality', 'Payment Typology 1',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs',
       'General_MDC']

subset_health_data = health_data[health_data_copy].copy()

In [ ]:
is_null = subset_health_data.isnull().sum()
is_null

Hospital County                        0
Facility Name                          0
Age Group                              0
Gender                                 0
Race                                   0
Ethnicity                              0
Length of Stay                         0
Type of Admission                      0
Patient Disposition                    0
APR Severity of Illness Description    0
APR Risk of Mortality                  0
Payment Typology 1                     0
Emergency Department Indicator         0
Total Charges                          0
Total Costs                            0
General_MDC                            0
dtype: int64

In [ ]:
subset_health_data.shape

(165681, 16)

In [ ]:
subset_health_data.head(5)

,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,APR Severity of Illness Description,APR Risk of Mortality,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs,General_MDC
0,Erie,Buffalo General,70 or Older,F,White,Not Span/Hispanic,8,Emergency,Specialized Hospitals,Extreme,Extreme,Medicare,True,55367.55,20121.21,Infectious Diseases
1,Erie,Buffalo General,50 to 69,M,Other Race,Spanish/Hispanic,2,Urgent,Home Care,Moderate,Minor,Miscellaneous/Other,False,48328.30,16943.32,Musculoskeletal System
2,Erie,Buffalo General,70 or Older,M,White,Not Span/Hispanic,1,Elective,Home Care,Minor,Minor,Medicare,False,17052.12,7673.72,Nervous System
3,Erie,Buffalo General,50 to 69,F,Black/African American,Not Span/Hispanic,6,Emergency,Home Care,Major,Moderate,Medicare,True,17287.12,9058.50,Digestive System
4,Erie,Buffalo General,30 to 49,M,White,Not Span/Hispanic,1,Emergency,Home Care,Moderate,Minor,Private Health Insurance,True,6821.06,1987.18,Digestive System


# We want APR Severity of Illness Description and APR Risk of Mortality to be ordinally encoded since these are orders: minor, moderate, major, and extreme.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinal_cols = ['APR Severity of Illness Description', 'APR Risk of Mortality']
severity_order = ['Minor', 'Moderate', 'Major', 'Extreme']
mortality_order = ['Minor', 'Moderate', 'Major', 'Extreme']

severity_encoder = OrdinalEncoder(categories=[severity_order])
mortality_encoder = OrdinalEncoder(categories=[mortality_order])

#fitting and transform to the ordinal encoding
subset_health_data['APR Severity of Illness Label'] = severity_encoder.fit_transform(subset_health_data[['APR Severity of Illness Description']])
subset_health_data['APR Risk of Mortality Label'] = mortality_encoder.fit_transform(subset_health_data[['APR Risk of Mortality']])

# After checking the ordinal encoding I am going to drop the original 2 columns (APR Severity of Illness Description and APR Risk of Mortality)

In [ ]:
dropping = ['APR Severity of Illness Description', 'APR Risk of Mortality']
subset_health_data.drop(columns=dropping, inplace=True)

In [ ]:
subset_health_data.head()

,Hospital County,Facility Name,Age Group,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Payment Typology 1,Emergency Department Indicator,Total Charges,Total Costs,General_MDC,APR Severity of Illness Label,APR Risk of Mortality Label
0,Erie,Buffalo General,70 or Older,F,White,Not Span/Hispanic,8,Emergency,Specialized Hospitals,Medicare,True,55367.55,20121.21,Infectious Diseases,3.0,3.0
1,Erie,Buffalo General,50 to 69,M,Other Race,Spanish/Hispanic,2,Urgent,Home Care,Miscellaneous/Other,False,48328.30,16943.32,Musculoskeletal System,1.0,0.0
2,Erie,Buffalo General,70 or Older,M,White,Not Span/Hispanic,1,Elective,Home Care,Medicare,False,17052.12,7673.72,Nervous System,0.0,0.0
3,Erie,Buffalo General,50 to 69,F,Black/African American,Not Span/Hispanic,6,Emergency,Home Care,Medicare,True,17287.12,9058.50,Digestive System,2.0,1.0
4,Erie,Buffalo General,30 to 49,M,White,Not Span/Hispanic,1,Emergency,Home Care,Private Health Insurance,True,6821.06,1987.18,Digestive System,1.0,0.0


# Making the ordianl values for these two columns an int.

In [ ]:
subset_health_data['APR Severity of Illness Label'] = subset_health_data['APR Severity of Illness Label'].astype(int)
subset_health_data['APR Risk of Mortality Label'] = subset_health_data['APR Risk of Mortality Label'].astype(int)

In [ ]:
subset_health_data.dtypes

Hospital County                    object
Facility Name                      object
Age Group                          object
Gender                             object
Race                               object
Ethnicity                          object
Length of Stay                      int64
Type of Admission                  object
Patient Disposition                object
Payment Typology 1                 object
Emergency Department Indicator       bool
Total Charges                     float64
Total Costs                       float64
General_MDC                        object
APR Severity of Illness Label       int64
APR Risk of Mortality Label         int64
dtype: object

# One Hot Encoding the rest of the object columns.

In [ ]:
object_columns = ['Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race', 'Ethnicity',
                  'Type of Admission', 'Patient Disposition', 'Payment Typology 1', 'General_MDC']

health_ohe = pd.get_dummies(subset_health_data, columns=object_columns)

In [ ]:
health_ohe

,Length of Stay,Emergency Department Indicator,Total Charges,Total Costs,APR Severity of Illness Label,APR Risk of Mortality Label,Hospital County_Allegany,Hospital County_Cattaraugus,Hospital County_Chautauqua,Hospital County_Erie,...,General_MDC_Metabolic Disorders,General_MDC_Musculoskeletal System,General_MDC_Myeloproliferative Diseases,General_MDC_Neonatal Disorders,General_MDC_Nervous System,General_MDC_Renal and Urinary Disorders,General_MDC_Respiratory System,General_MDC_Skin and Breast Disorders,General_MDC_Substance-induced Disorders,General_MDC_Trauma
0,8,True,55367.55,20121.21,3,3,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,2,False,48328.30,16943.32,1,0,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
2,1,False,17052.12,7673.72,0,0,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
3,6,True,17287.12,9058.50,2,1,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,1,True,6821.06,1987.18,1,0,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165676,2,False,2452.56,3887.06,0,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
165677,2,False,2621.43,3992.01,0,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
165678,2,False,2390.95,3860.08,0,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
165679,1,False,1338.51,2044.00,0,0,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False


# Modeling: Predicting Length of Stay

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
import numpy as np

# Linear Regression

In [ ]:
X = health_ohe.drop(columns=['Length of Stay'])
y = health_ohe['Length of Stay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

mae_lr = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae_lr)

Mean Absolute Error: 2.3771701780901235


In [ ]:
#MSE
mse_lr = mean_squared_error(y_test, y_pred)

#RMSE
rmse_lr = np.sqrt(mse_lr)

#R squared
r2_lr = r2_score(y_test, y_pred)

#MAPE
mape_lr = mean_absolute_percentage_error(y_test, y_pred)


print("Mean Squared Error (MSE):", mse_lr)
print("Root Mean Squared Error (RMSE):", rmse_lr)
print("R Squared:", r2_lr)
print("Mean Absolute Percent Error:", mape_lr)

Mean Squared Error (MSE): 23.627992217817482
Root Mean Squared Error (RMSE): 4.860863320215605
R Squared: 0.6012544017858414
Mean Absolute Percent Error: 0.6980518498089506


# Random Forest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


rand_for = RandomForestRegressor(n_estimators=100, random_state=42)

rand_for.fit(X_train, y_train)

y_pred = rand_for.predict(X_test)

mae_ran = mean_absolute_error(y_test, y_pred)
rmse_ran = mean_squared_error(y_test, y_pred, squared=False)
r2_ran = r2_score(y_test, y_pred)
mape_ran = mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae_ran)
print("Root Mean Squared Error (RMSE):", rmse_ran)
print("R Squared:", r2_ran)

Mean Absolute Error (MAE): 1.0988321564012031
Root Mean Squared Error (RMSE): 2.8610903487288155
R Squared: 0.8618559362318958


# XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X_train.columns = [str(col).replace('[', '').replace('/', '').replace(']', '').replace('<', '') for col in X_train.columns]
X_test.columns = [str(col).replace('[', '').replace('/', '').replace(']', '').replace('<', '') for col in X_test.columns]

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_reg.fit(X_train, y_train)

y_pred_xgb = xgb_reg.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
r2_xgb = r2_score(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred)

print("XGBoost - Mean Absolute Error (MAE):", mae_xgb)
print("XGBoost - Root Mean Squared Error (RMSE):", rmse_xgb)
print("XGBoost - R Squared (R2):", r2_xgb)
print("XGBoost - Mean Absolute Percent Error (MAPE):", mape_xgb)


XGBoost - Mean Absolute Error (MAE): 1.2294831656032648
XGBoost - Root Mean Squared Error (RMSE): 2.974225529351345
XGBoost - R Squared (R2): 0.8507147578553315
XGBoost - Mean Absolute Percent Error (MAPE): 0.2601880978195484


In [ ]:
#metrics for models for length of stay
metrics_df_los = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'XGBoost'],
    'MAE': [mae_lr, mae_ran, mae_xgb],
    'RMSE': [rmse_lr, rmse_ran, rmse_xgb],
    'MAPE': [mape_lr, mape_ran, mape_xgb],
    'R^2': [r2_lr, r2_ran, r2_xgb]
})

metrics_df_los.set_index('Model', inplace=True)

print(metrics_df_los)

                        MAE      RMSE      MAPE       R^2
Model                                                    
Linear Regression  2.377170  4.860863  0.698052  0.601254
Random Forest      1.098832  2.861090  0.260188  0.861856
XGBoost            1.229483  2.974226  0.260188  0.850715


# Modeling: Predicting Total Costs

# Linear Regression

In [ ]:
X_cost = health_ohe.drop(columns=['Total Costs'])
y_cost = health_ohe['Total Costs']

X_train, X_test, y_train, y_test = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

Mean Absolute Error: 2274.690075641672


In [ ]:
#MSE
mse = mean_squared_error(y_test, y_pred)

#RMSE
rmse = np.sqrt(mse)

#R squared
r2 = r2_score(y_test, y_pred)

#MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)


print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R Squared:", r2)
print("Mean Absolute Percent Error:", mape)

Mean Squared Error (MSE): 41779001.00021116
Root Mean Squared Error (RMSE): 6463.667766849651
R Squared: 0.9244709148501475
Mean Absolute Percent Error: 0.33041656342247244


# Based on the R Squared value, predicting Total Costs does better than predicting Length of Stay.

# Random Forest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cost, y_cost, test_size=0.2, random_state=42)


rand_for = RandomForestRegressor(n_estimators=100, random_state=42)

rand_for.fit(X_train, y_train)

y_pred = rand_for.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R Squared:", r2)

Mean Absolute Error (MAE): 968.9075670043198
Root Mean Squared Error (RMSE): 5015.079991646522
R Squared: 0.9545313665052373


# Higher R Sqaured indicates that random forest predicts the cost better than linear regression